https://medium.com/@bavalpreetsinghh/llamaindex-chunking-strategies-for-large-language-models-part-1-ded1218cfd30

In [1]:
from src.rag.embedding_generator import docs_parse,generate_embeddings
from src.rag.data_to_milvus import load_nodes_to_milvus
from src.rag.data_to_milvus import create_milvus_collection, insert_embeddings_to_milvus
from src.config import config
import os

/opt/homebrew/anaconda3/envs/finrag/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Load Markdown files
md_files = []
md_files_dir = config.get_path('dataset', 'preprocessed_md')
for root, dirs, files in os.walk(md_files_dir):
    for file in files:
        if file.endswith('.md'):
            md_files.append(os.path.join(root, file))

len(md_files)
# md_file = '/Users/swgj/Documents/Code/python_project/FinanceRAG/data/markdown/0b46f7a2d67b5b59ad67cafffa0e12a9f0837790/0b46f7a2d67b5b59ad67cafffa0e12a9f0837790.md'
# text, doc_ids = load_texts(md_fir)

80

In [3]:
md_files[15]

'/Users/swgj/Documents/Code/python_project/FinanceRAG/data/preprocessed_md/4170a3382f5e92d32890780d79cb39115a756f40.md'

In [4]:
nodes = docs_parse(md_files)

In [5]:
len(nodes)

23317

In [6]:
# for node in nodes:
#     if node.id_ == '0934726b-2796-4725-bdcd-6faa389ed2a7':
#         print(dict(node))

In [7]:
dict(nodes[10])

{'id_': '4c7608c6-65cb-4752-b529-24b6cb38a771',
 'embedding': None,
 'metadata': {'filename': 'c48b454b4d171ba375fb88a6469b0a1d7f4625c2.md',
  'extension': '.md',
  'Header_1': '浙江金卡高科技股份有限公司',
  'Header_2': '浙江金卡高科技股份有限公司',
  'Header_6': 'Zhejiang Goldcard High-Tech Co.，Ltd.'},
 'excluded_embed_metadata_keys': [],
 'excluded_llm_metadata_keys': [],
 'relationships': {<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='1c980d98-1e4d-4f08-a376-04e2ffeb0b06', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'filename': 'c48b454b4d171ba375fb88a6469b0a1d7f4625c2.md', 'extension': '.md'}, hash='bdc040de93a42ae1549ca371fc7fa3266cb4fa4725dc44b1409647b820d29c09'),
  <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='bb7a4852-0b09-4e8a-8cf0-0923505c32c5', node_type=<ObjectType.TEXT: '1'>, metadata={'filename': 'c48b454b4d171ba375fb88a6469b0a1d7f4625c2.md', 'extension': '.md', 'Header_1': '浙江金卡高科技股份有限公司', 'Header_2': '浙江金卡高科技股份有限公司'}, hash='897c1b0c509231a5d452a9b5740ab92537848441034

In [8]:
index = load_nodes_to_milvus(nodes)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
Settings.llm = Ollama(model="qwen2.5")
query_engine = index.as_chat_engine()

In [10]:
res = query_engine.query("中国铁路通信信号有限公司发行前每股净资产为多少？")

In [11]:
print(res)

中国铁路通信信号有限公司发行前每股净资产为3.29元。


In [12]:
print(dict(res.source_nodes[0]))

{'node': TextNode(id_='a9df9d69-4200-4e19-8f38-9818c413dca5', embedding=None, metadata={'filename': '4170a3382f5e92d32890780d79cb39115a756f40.md', 'extension': '.md', 'Header_1': '中国铁路通信信号股份有限公司', 'Header_2': '首次公开发行股票并在科创板上市', 'Header_3': '第二节 概 览', 'Header_5': '三、发行人报告期主要财务数据和财务指标'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b5edb28b-9afa-4653-a904-ced4fe9b6495', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'filename': '4170a3382f5e92d32890780d79cb39115a756f40.md', 'extension': '.md'}, hash='14b8dbb4b11ec4c7c00e1076e5dbeaa77d164380c76a8a277f81b9b13533489a'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='9990f677-8217-4bda-89c8-6290be7ef10f', node_type=<ObjectType.TEXT: '1'>, metadata={'filename': '4170a3382f5e92d32890780d79cb39115a756f40.md', 'extension': '.md', 'Header_1': '中国铁路通信信号股份有限公司', 'Header_2': '首次公开发行股票并在科创板上市', 'Header_3': '第二节 概 览', 'Header_5': '二、本次发行概况'}, hash='

In [22]:
from llama_index.vector_stores.deeplake import DeepLakeVectorStore
from llama_index.vector_stores.milvus import MilvusVectorStore


In [15]:
index.storage_context.persist(persist_dir='data/milvus')

In [20]:
DeepLakeVectorStore('data/milvus_deep')

DeepLakeVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=True, ingestion_batch_size=1024, num_workers=4, token=None, read_only=False, dataset_path='data/milvus_deep')

In [21]:
# pkl save
import pickle
with open('data/milvus.pkl', 'wb') as f:
    pickle.dump(index, f)

In [1]:
from pymilvus import MilvusClient
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core import StorageContext

# initialize client
client = MilvusClient(uri='data/milvus_md.db')
# db = chromadb.PersistentClient(path="./chroma_db")



In [2]:
# get collection
# chroma_collection = db.get_or_create_collection("quickstart")
# collection = client.get(collection_name='llamacollection')

from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from src.rag.data_to_milvus import embedding_model
from src.config import config

Settings.llm = Ollama(model="qwen2.5")
Settings.embed_model = embedding_model(config.get_path('models','embedding'))
# query_engine = index.as_chat_engine()

# assign chroma as the vector_store to the context
vector_store = MilvusVectorStore(uri='data/milvus_md.db')
# vector_store = MilvusVectorStore(uri='data/milvus_md.db', collection_name='llamacollection')
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# load your index from stored vectors
index_new = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

In [15]:
# import pickle
# with open('data/milvus_new.pkl', 'wb') as f:
#     pickle.dump(index_new, f)


In [3]:
# create a query engine
query_engine = index_new.as_query_engine()
# response = query_engine.query("介绍一下武汉力源信息技术股份有限公司?")
response = query_engine.query("中国铁路通信信号有限公司发行前每股净资产为多少？")
print(response)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


中国铁路通信信号有限公司发行前每股净资产为3.29元。


In [4]:
response.get_formatted_sources()

'> Source (Doc id: a9df9d69-4200-4e19-8f38-9818c413dca5): 三、发行人报告期主要财务数据和财务指标\n\n**2018 年** **12 月** **31 日** **2017 年** **12 月** **31 日** **2016 年** **12 月*...\n\n> Source (Doc id: cf24631d-cf4e-422d-99ef-76b174699e61): 十一、发行人主要财务指标\n\n（一）基本财务指标\n\n**2018 年** **2017 年** **2016 年**\n项目\n\n**12 月** **31 日** **12 月** **31 日**...'

In [5]:
res = query_engine.query("中国铁路通信信号有限公司的介绍")
print(res)

中国铁路通信信号股份有限公司主要从事轨道交通控制系统的设计集成、设备制造及系统交付。公司致力于提供CTCS列车运行控制系统、城市轨道交通CBTC系统、货运铁路CIPS综合自动化系统等的研发与应用，并且在国内市场积累了丰富的建设经验，持续拓展海外市场。

截至信息提供的日期，公司在技术创新方面取得了显著成就，拥有6个省部级重点实验室、工程技术研究中心，1个国家企业技术中心，1个国家工业设计中心，15个省级企业技术中心，以及6个院士专家工作站和3个博士后科研流动工作站。这些平台和技术资源为公司的研发工作提供了坚实的支持。

凭借强大的研发实力和技术积累，中国铁路通信信号股份有限公司在铁路、城市轨道交通等领域取得了多项重要科技成果，并将继续加大投入，利用如北斗定位、人工智能、下一代移动通信、大数据等前沿科技，进一步提升其在列车运行控制核心技术、轨道交通智能运维、智慧城市及行业通信信息领域的竞争力。

主要竞争对手包括铁科院和和利时。这些信息展示了公司在市场和技术方面的优势及其面临的竞争环境。


In [8]:
len(res.source_nodes)

2